In [79]:
import numpy as np
import pandas as pd
from my_stuff import MufexKeys
from quantfreedom.enums import *
from quantfreedom.exchanges.mufex_exchange.mufex import Mufex

mufex_main = Mufex(
    api_key=MufexKeys.api_key,
    secret_key=MufexKeys.secret_key,
    use_test_net=False,
)

%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [80]:
candles = mufex_main.get_candles(symbol="BTCUSDT", timeframe='5m', candles_to_dl=200)

It took 00 mins and 00 seconds to download 200 candles


In [81]:
from quantfreedom.indicators.indicators import qf_calc_rsi


qf_calc_rsi(prices=candles[:,4], period=14)

array([        nan,         nan,         nan,         nan,         nan,
               nan,         nan,         nan,         nan,         nan,
               nan,         nan,         nan,         nan, 50.66965651,
       48.44033627, 49.50795376, 51.22190415, 40.19023906, 38.36986082,
       43.19331973, 46.36569862, 46.47793139, 41.8950945 , 44.66881562,
       30.15363247, 33.58630455, 33.83653976, 36.15761727, 28.76216061,
       31.03587102, 22.76710551, 25.10317051, 32.81599483, 38.79818459,
       38.7445485 , 37.81264031, 43.26943896, 36.98273314, 55.18963596,
       45.33199097, 39.14858981, 49.59195991, 48.77896862, 52.22905356,
       64.82391982, 66.6032136 , 59.80370635, 62.23518723, 60.75581879,
       58.77033505, 57.5981188 , 58.85666939, 58.84954948, 63.46983242,
       68.38772316, 62.29664511, 61.65307281, 60.72272658, 49.46834615,
       39.44645855, 39.8066511 , 29.23210542, 27.18064263, 27.5968765 ,
       25.99473006, 25.49323678, 17.46640046, 15.95080363, 16.96

In [82]:
(26.50-26.0)/26.0

0.019230769230769232

In [83]:
close_shift = np.roll(candles[:,4], 1)
close_shift[0] = np.nan

In [84]:
closes = candles[:,4]

In [85]:
close_shift

array([    nan, 34124.3, 34153.8, 34138.3, 34150. , 34172.5, 34184.4,
       34140.1, 34142.3, 34120.1, 34123.4, 34129.3, 34165.4, 34174.8,
       34151.9, 34127.7, 34145.9, 34135.8, 34156.1, 34120. , 34124. ,
       34108.7, 34124.5, 34102.9, 34080.9, 34099.9, 34051.2, 34075.4,
       34054.8, 34049.9, 34022.8, 34027.8, 33993.3, 33985.1, 34020.1,
       34039.9, 34055.4, 34026.3, 34039.7, 34020.2, 34084.4, 34043.7,
       33969.8, 34045.8, 34011. , 34049.9, 34166.7, 34181.6, 34137.6,
       34196.3, 34190.7, 34140.1, 34136.7, 34130.9, 34195.2, 34204. ,
       34172.8, 34164.8, 34124.8, 34152.2, 34162.5, 34090.6, 34050.6,
       34038.7, 34004.6, 33960.4, 33932.4, 33918.3, 33940.2, 33941.9,
       33933.9, 33921.9, 33913.3, 33873.6, 33920.9, 33924.5, 33940.3,
       33941.7, 33947.8, 34002.7, 34027.5, 34117.1, 33975.9, 33973.8,
       34010.9, 33975.4, 33994. , 34009.5, 34004.5, 33984.5, 33993.3,
       33948.8, 33940.8, 33925.5, 33879.3, 33874.9, 33881. , 33859.3,
       33646. , 3365

In [86]:
pchg = (closes-close_shift)/close_shift

In [87]:
pchg

array([            nan,  8.64486598e-04, -4.53829442e-04,  3.42723569e-04,
        6.58857980e-04,  3.48233228e-04, -1.29591276e-03,  6.44403502e-05,
       -6.50219815e-04,  9.67171843e-05,  1.72901880e-04,  1.05774217e-03,
        2.75132151e-04, -6.70084390e-04, -7.08598936e-04,  5.33291139e-04,
       -2.95789538e-04,  5.94683587e-04, -1.05691224e-03,  1.17233294e-04,
       -4.48364787e-04,  4.63224925e-04, -6.32976307e-04, -6.45106428e-04,
        5.57497014e-04, -1.42815668e-03,  7.10694484e-04, -6.04541693e-04,
       -1.43885737e-04, -7.95890737e-04,  1.46960274e-04, -1.01387689e-03,
       -2.41224006e-04,  1.02986309e-03,  5.82008871e-04,  4.55347989e-04,
       -8.54490037e-04,  3.93813021e-04, -5.72860513e-04,  1.88711413e-03,
       -1.19409466e-03, -2.17073937e-03,  2.23728135e-03, -1.02215251e-03,
        1.14374761e-03,  3.43025971e-03,  4.36097136e-04, -1.28724226e-03,
        1.71951162e-03, -1.63760407e-04, -1.47993460e-03, -9.95896321e-05,
       -1.69905117e-04,  

In [88]:
gain = np.where(pchg > 0, pchg, 0)
loss = np.where(pchg < 0, abs(pchg), 0)
gain[0] = np.nan
loss[0] = np.nan
alpha = 1/14
rma_up = np.zeros_like(gain)
rma_down = np.zeros_like(gain)
avg_gain = np.full_like(gain, np.nan)
avg_loss = np.full_like(gain, np.nan)

In [89]:
for i in range(1, gain.size):
    rma_up[i] = alpha * gain[i] + (1 - alpha) * rma_up[i-1]
    rma_down[i] = alpha * loss[i] + (1 - alpha) * rma_down[i-1]

In [90]:
rma_up[:14] = 0
rma_down[:14] = 0

In [91]:
for i in range(14, gain.size):
    avg_gain[i] = rma_up[i-14:i].mean()
    avg_loss[i] = rma_down[i-14:i].mean()

In [92]:
avg_gain

array([           nan,            nan,            nan,            nan,
                  nan,            nan,            nan,            nan,
                  nan,            nan,            nan,            nan,
                  nan,            nan, 0.00000000e+00, 1.16943633e-05,
       2.52742880e-05, 3.78842181e-05, 5.26275389e-05, 6.63177653e-05,
       7.96282475e-05, 9.19879810e-05, 1.05828269e-04, 1.18679965e-04,
       1.30613682e-04, 1.44539364e-04, 1.57470354e-04, 1.73103694e-04,
       1.87620367e-04, 1.89405772e-04, 1.88342774e-04, 1.88105503e-04,
       1.84851079e-04, 1.81829114e-04, 1.83679278e-04, 1.88366721e-04,
       1.92679158e-04, 1.96683563e-04, 2.02411190e-04, 2.04885328e-04,
       2.16810876e-04, 2.24258606e-04, 2.31174355e-04, 2.49010823e-04,
       2.65573258e-04, 2.86038311e-04, 3.22542900e-04, 3.58665004e-04,
       3.86952553e-04, 4.19023149e-04, 4.46479784e-04, 4.71975231e-04,
       4.93640325e-04, 5.13757912e-04, 5.32422248e-04, 5.51066409e-04,
      

In [93]:
rs = avg_gain / avg_loss

In [94]:
100-(100/(1+rs))

array([        nan,         nan,         nan,         nan,         nan,
               nan,         nan,         nan,         nan,         nan,
               nan,         nan,         nan,         nan,         nan,
       46.37481306, 49.21603255, 49.11563588, 50.55266241, 49.29152266,
       48.73105669, 47.78817674, 47.69952865, 47.02635047, 45.99296675,
       45.71291361, 44.56134985, 44.21123279, 43.62700257, 42.91614832,
       41.56240702, 40.59505163, 38.91653642, 37.75326351, 37.29992447,
       37.43659292, 37.5534936 , 37.57675731, 38.06607123, 37.97414225,
       39.48290715, 40.01086084, 39.92362359, 41.02695152, 41.88794515,
       43.06342844, 45.77592514, 48.2654686 , 49.5985719 , 51.07640226,
       52.14417233, 52.86893826, 53.36043488, 53.90300798, 54.37827123,
       55.1913346 , 56.27409888, 56.70708984, 57.06595645, 57.32514968,
       56.88509938, 55.75134811, 54.71685902, 53.22461259, 51.57053795,
       50.0574857 , 48.43804218, 46.85903337, 45.01587328, 43.20